In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [15]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [34]:
soup = BeautifulSoup(source, 'html.parser')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [36]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [37]:
print(My_table.tr.text)


Postal Code

Borough

Neighborhood



In [40]:
df=df[df['Borough']!='Not assigned']


In [43]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [44]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [45]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [46]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [47]:
df_merge.drop_duplicates(inplace=True)

In [48]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)


In [49]:
df_merge.head()


,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
8,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [50]:
df_merge.shape

(103, 3)

In [51]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [52]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [53]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [55]:
geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,43.770992,-79.216917,Scarborough,"Woburn, Woburn"
4,M1H,43.773136,-79.239476,Scarborough,"Cedarbrae, Cedarbrae"


In [56]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [57]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
43,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
44,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790


In [58]:
CLIENT_ID = '3HDEN4DXK5E5XJMNX4PHBEFVZJVSSOLZYITEXMIVE5O0XPKB' # your Foursquare ID
CLIENT_SECRET = 'K5FGUIZ4EMILD3LT5WV42TZ1UIX2X2AO1BNY214G1BSYM52U' # your Foursquare Secret
VERSION = '20180604'


In [59]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches, The Beaches
The Danforth West, Riverdale, The Danforth West, Riverdale
India Bazaar, The Beaches West, India Bazaar, The Beaches West
Studio District, Studio District
Lawrence Park, Lawrence Park
Davisville North, Davisville North
North Toronto West,  Lawrence Park, North Toronto West,  Lawrence Park
Davisville, Davisville
Moore Park, Summerhill East, Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park, Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale, Rosedale
St. James Town, Cabbagetown, St. James Town, Cabbagetown
Church and Wellesley, Church and Wellesley
Regent Park, Harbourfront, Regent Park, Harbourfront
Garden District, Ryerson, Garden District, Ryerson
St. James Town, St. James Town
Berczy Park, Berczy Park
Central Bay Street, Central Bay Street
Richmond, Adelaide, King, Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands, Harbourfront East, Union Station, Toronto Islands
To

In [61]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Beaches, The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"The Beaches, The Beaches",43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,"The Beaches, The Beaches",43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,"The Beaches, The Beaches",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [62]:

toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Berczy Park, Berczy Park",57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place, Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport, CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
"Central Bay Street, Central Bay Street",68,68,68,68,68,68
"Christie, Christie",17,17,17,17,17,17
"Church and Wellesley, Church and Wellesley",70,70,70,70,70,70
"Commerce Court, Victoria Hotel, Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Davisville North, Davisville North",7,7,7,7,7,7
